GAN의 아이디어에 대해 이해했다면, 이제 본격적으로 GAN을 파헤쳐볼 차례다. 본 포스팅에서는 우리에게 가장 익숙한 mnist 이미지를 생성하는 모델을 기준으로 예시를 들겠다. 먼저 이미지를 판별하는 Discriminator(이하 D)는 CNN 판별기처럼 구성할 수 있다. 그래서 이해하기에 매우 쉽다. 하지만 Generator(이하 G)를 구성하는 것은 새로운 아이디어가 도입된다.

+ G는 random한 noise를 생성해내는 vector z를 input으로 하며

+ D가 판별하고자 하는 input image(여기서는 28X28의 mnist 이미지)를 output으로 하는 neural network unit이라고 할 수 있다

이렇게 GAN의 코어가 되는 모델은 D와 G 두 가지이다. 학습 과정에서는 실제 mnist 이미지, Real Image를 D로 하여금 '진짜'라고 학습시키는 1번 과정, 그리고 vector z와 G에 의해 생성된 Fake Image를 '가짜'라고 학습시키는 2번과정으로 나뉜다. 여기서 유의할 점은 D가 두번 학습되고 G는 1번 학습되는 것이 아니라, 1번 과정에서의 Real Image와 Fake Image를 D의 x input으로 합쳐서 학습한다는 것이다.


In [46]:
#실행할필요없음

def train_D(self):
  """
  train Discriminator
  """

  # Real data
  real = self.data.get_real_sample()

  # Generated data
  z = self.data.get_z_sample(self.batch_size)
  generated_images = self.gan.G.predict(z)

  # labeling and concat generated, real images
  x = np.concatenate((real, generated_images), axis=0)
  y = [0.9] * self.batch_size + [0] * self.batch_size

  # train discriminator
  self.gan.D.trainable = True
  loss = self.gan.D.train_on_batch(x, y)

  return loss

def train_G(self):
  """
  train Generator
  """

  # Generated data
  z = self.data.get_z_sample(self.batch_size)

  # labeling
  y = [1] * self.batch_size

  # train generator
  self.gan.D.trainable = False
  loss = self.gan.GD.train_on_batch(z, y)
  return loss

+ train_D는 D를 학습하는 부분

+  train_G는 D(G(z))에서 G를 학습하는 부분

D.trainable을 사용하여, 위에서 설명한 대로 D는 한 번만 학습되도록 구현하였다. 코드에서 D(G(z))에서 D의 학습을 False로 한다면, 결국 G만 학습이 된다. 눈여겨 보아야 할 부분은 'x = np.concatenate((real, generated_images), axis=0)' 이다. 이 부분을 통해 진짜이미지와 가짜이미지를 D에게 한번에 학습시킨다.

GAN 프레임 워크는 코어가 되는 두 개의 모델의 학습에 따라 진행된다. D의 목표는 Real, 혹은 Fake 이미지를 제대로 분류해내는 것이다. 그리고 G의 임무는 완벽하게 D가 틀리도록 하는 것이다. 그래서 두 코어 모델의 Loss 지표는 반대가 되며, 이 때문에도 '적대적' 모델로 불린다.

$$min_G max_D V(D,G) = E_{x\tilde{}p_{data}(x)}[logD(x)]+E_{z\tilde{}p_z (z)}[log(1-D(G(z)))]$$

https://arxiv.org/pdf/1406.2661.pdf

GAN 모델은 일반적인 머신 러닝, 혹은 딥 러닝 모델과는 달리 명확한 평가의 기준이 없다. Loss는 단지 학습을 위한 오토 파라미터의 구실을 하는 셈이고, 실제적인 Loss를 나타내거나 Accuracy와 같은 기준이 되는 명확한 평가지표가 존재하지 않는다. 이미지를 생성하는 GAN의 경우, 사람의 육안으로 결과물을 평가할 수 있을 뿐이다.


Loss함수를 정의하고 이를 최적화 할때, 실제 환경에서는 생각보다 G의 초기 성능이 안좋다. 그래서 D(G(z))가 0에 가깝게 되는데, 원론적인 수식으로 적용하면 학습이 잘 안된다. 그래서 약간의 테크닉을 써서 수식을 살짝 바꾼다. (역시 수식에 대한 자세한 내용, 연구적인 내용에 관심이 있는 사람이라면 논문을 참고하자) 또한 D : G의 학습 비율은 1 : 5 와 같은 형태로 불균형하게 하는것이 일반적인듯 하다. D가 G에 비해 너무 정확하다면, G의 gradient가 vanishing되는 문제가 생기기도 하고, 반대의 경우도 생긴다. 어쨌든 이러한 문제점을 해결하기 위해 학습 비율을 조정을 하면 해결된다고 한다. 실제 코드를 돌려보면, D, G의 iterate를 다른 비율로 학습하는 것이 훨씬 학습이 잘된다.


GAN은 D, G 그리고 Noise에 대한 함수와 네트워크 구성을 자유롭게 하면서, GAN의 advanced 모델들을 구현해보는 쏠쏠한 재미가 있다. 그러한 모델들을 구현해보기 전에, 우선 기초가 되는 Gaussian 분포 생성과 mnist 이미지 생성에 대한 튜토리얼을 진행해보는 것을 권장한다.

In [1]:
#import argparse
#jupyter에서 argparse쓰면 에러난다!!!
import easydict

import numpy as np

from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, MaxPooling2D, UpSampling2D
#from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
from keras.datasets import mnist
#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras import initializers
from keras import backend as K


K.set_image_data_format('channels_first')


class Data:
    """
    Define dataset for training GAN
    """
    def __init__(self, batch_size, z_input_dim):
        # load mnist dataset
        # 이미지는 보통 -1~1 사이의 값으로 normalization : generator의 outputlayer를 tanh로
        (X_train, y_train), (X_test, y_test) = mnist.load_data()
        self.x_data = ((X_train.astype(np.float32) - 127.5) / 127.5)
        self.x_data = self.x_data.reshape((self.x_data.shape[0], 1) + self.x_data.shape[1:])
        self.batch_size = batch_size
        self.z_input_dim = z_input_dim

    def get_real_sample(self):
        """
        get real sample mnist images

        :return: batch_size number of mnist image data
        """
        return self.x_data[np.random.randint(0, self.x_data.shape[0], size=self.batch_size)]

    def get_z_sample(self, sample_size):
        """
        get z sample data

        :return: random z data (batch_size, z_input_dim) size
        """
        return np.random.uniform(-1.0, 1.0, (sample_size, self.z_input_dim))


class GAN:
    #predict 에러 해결
    def predict(self, x):
        x = self.forward(x)
        return x

    def __init__(self, learning_rate, z_input_dim):
        """
        init params

        :param learning_rate: learning rate of optimizer
        :param z_input_dim: input dim of z
        """
        self.learning_rate = learning_rate
        self.z_input_dim = z_input_dim
        self.D = self.discriminator()
        self.G = self.generator()
        self.GD = self.combined()

    def discriminator(self):
        """
        define discriminator
        """
        D = Sequential()
        D.add(Conv2D(256, (5, 5),
                     padding='same',
                     input_shape=(1, 28, 28),
                     kernel_initializer=initializers.RandomNormal(stddev=0.02)))
        D.add(LeakyReLU(0.2))
        D.add(MaxPooling2D(pool_size=(2, 2), strides=2))
        D.add(Dropout(0.3))
        D.add(Conv2D(512, (5, 5), padding='same'))
        D.add(LeakyReLU(0.2))
        D.add(MaxPooling2D(pool_size=(2, 2), strides=2))
        D.add(Dropout(0.3))
        D.add(Flatten())
        D.add(Dense(256))
        D.add(LeakyReLU(0.2))
        D.add(Dropout(0.3))
        D.add(Dense(1, activation='sigmoid'))

        adam = Adam(lr=self.learning_rate, beta_1=0.5)
        D.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
        return D

    def generator(self):
        """
        define generator
        """
        G = Sequential()
        G.add(Dense(512, input_dim=self.z_input_dim))
        G.add(LeakyReLU(0.2))
        G.add(Dense(128 * 7 * 7))
        G.add(LeakyReLU(0.2))
        G.add(BatchNormalization())
        G.add(Reshape((128, 7, 7), input_shape=(128 * 7 * 7,)))
        G.add(UpSampling2D(size=(2, 2)))
        G.add(Conv2D(64, (5, 5), padding='same', activation='tanh'))
        G.add(UpSampling2D(size=(2, 2)))
        G.add(Conv2D(1, (5, 5), padding='same', activation='tanh'))

        adam = Adam(lr=self.learning_rate, beta_1=0.5)
        G.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
        return G

    def combined(self):
        """
        defien combined gan model
        """
        G, D = self.G, self.D
        D.trainable = False
        GD = Sequential()
        GD.add(G)
        GD.add(D)

        adam = Adam(lr=self.learning_rate, beta_1=0.5)
        GD.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
        D.trainable = True
        return GD


class Model:
    #predict 에러 해결
    def predict(self, x):
        x = self.forward(x)
        return x

    def __init__(self, args):
        self.epochs = args.epochs
        self.batch_size = args.batch_size
        self.learning_rate = args.learning_rate
        self.z_input_dim = args.z_input_dim
        self.data = Data(self.batch_size, self.z_input_dim)

        # the reason why D, G differ in iter : Generator needs more training than Discriminator
        self.n_iter_D = args.n_iter_D
        self.n_iter_G = args.n_iter_G
        self.gan = GAN(self.learning_rate, self.z_input_dim)
        self.d_loss = []
        self.g_loss = []

        # print status
        batch_count = self.data.x_data.shape[0] / self.batch_size
        print('Epochs:', self.epochs)
        print('Batch size:', self.batch_size)
        print('Batches per epoch:', batch_count)
        print('Learning rate:', self.learning_rate)
        print('Image data format:', K.image_data_format())

    def fit(self):
        for epoch in range(self.epochs):

            # train discriminator by real data
            dloss = 0
            for iter in range(self.n_iter_D):
                dloss = self.train_D()

            # train GD by generated fake data
            gloss = 0
            for iter in range(self.n_iter_G):
                gloss = self.train_G()

            # print loss data
            print('Discriminator loss:', str(dloss))
            print('Generator loss:', str(gloss))

    def train_D(self):

        #train Discriminator

        # Real data
        real = self.data.get_real_sample()

        # Generated data
        z = self.data.get_z_sample(self.batch_size)
        generated_images = self.gan.G.predict(z)

        # labeling and concat generated, real images
        x = np.concatenate((real, generated_images), axis=0)
        y = [0.9] * self.batch_size + [0] * self.batch_size

        # train discriminator
        self.gan.D.trainable = True
        loss = self.gan.D.train_on_batch(x, y)
        return loss

    def train_G(self):

        #train Generator

        # Generated data
        z = self.data.get_z_sample(self.batch_size)

        # labeling
        y = [1] * self.batch_size

        # train generator
        self.gan.D.trainable = False
        loss = self.gan.GD.train_on_batch(z, y)
        return loss

def main():
    # set hyper parameters
 
    args = easydict.EasyDict({
        
        "batch_size": 128,
 
        "epochs": 200,
 
        "learning_rate": 0.0002,
 
        "z_input_dim": 100,
 
        "n_iter_D": 1,

        "n_iter_G": 5,
 
        "unit": 1000
 
      })

    
    """
    parser = argparse.ArgumentParser()
    parser.add_argument('--batch_size', type=int, default=128,
                        help='Batch size for networks')
    parser.add_argument('--epochs', type=int, default=200,
                        help='Epochs for the networks')
    parser.add_argument('--learning_rate', type=float, default=0.0002,
                        help='Learning rate')
    parser.add_argument('--z_input_dim', type=int, default=100,
                        help='Input dimension for the generator.')
    parser.add_argument('--n_iter_D', type=int, default=1,
                        help='training iteration for D')
    parser.add_argument('--n_iter_G', type=int, default=5,
                        help='training iteration for G')
    args = parser.parse_args()
    """

    # run model
    model = Model(args)
    model.fit()

main()

"""
if __name__ == '__main__':
    main()
"""

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epochs: 200
Batch size: 128
Batches per epoch: 468.75
Learning rate: 0.0002
Image data format: channels_first


UnimplementedError: ignored